# SMS Filter using  multinomial Naive Bayes algorithm

We'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can also download the dataset directly from this [link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on this [page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

Solutions to this project can be found at this [link](https://github.com/dataquestio/solutions/blob/master/Mission433Solutions.ipynb) or by clicking the key icon at the top right of the interface.


In [148]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Original data
We import the data and inspect the format and quality of it

In [149]:
sms_df_raw =  pd.read_csv('SMSSpamCollection', sep='\t', header=None)
sms_df_raw.shape

(5572, 2)

In [150]:
# Rename columns to represent correctly the attributes
sms_df = sms_df_raw.copy()
dic_col = {0:'Label', 1:'SMS'}
sms_df.rename(columns=dic_col,inplace=True)

In [151]:
sms_df.head()
sms_df.loc[:,'Label'].unique()

array(['ham', 'spam'], dtype=object)

The data seems alright, with 5572 sms as it was indicated in the daata source. The SMS message, however, seems rather cryptic. The label options are either 'ham' or 'spam'. The first indicates that is a real SMS, whereas the latter is a SPAM mesage.

In [152]:
sms_df.loc[:,'Label'].value_counts()/sms_df.loc[:,'Label'].value_counts().sum()

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

We see that a 86% of the SMS are 'ham' vs only 13% 'spam'.

## Test setup for algorithm
We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

* The training set will have 4,458 messages (about 80% of the dataset).
* The test set will have 1,114 messages (about 20% of the dataset).

In [153]:
sms_df = sms_df.sample(frac=1, random_state=1).reset_index(drop=True)
sms_train = sms_df.iloc[:4458,:]
sms_test = sms_df.iloc[4458:,:]

In [154]:
spam_perc_train = sms_train.loc[:,'Label'].value_counts()[1]/sms_train.loc[:,'Label'].value_counts().sum()
spam_perc_test = sms_test.loc[:,'Label'].value_counts()[1]/sms_test.loc[:,'Label'].value_counts().sum()
print('Spam percentage in train is {0}% and in test {1}%'.format(spam_perc_train*100, spam_perc_test*100))

Spam percentage in train is 13.458950201884253% and in test 13.195691202872531%


The percentages in both seem fairly close. This means that the test sample should be large enough for our purpose.

## Data cleaning
We proceed to do the following operations:
* Remove punctuation or non-word characters
* Transform the text into lower case
* Transform the data from Label, SMS -> Label, w1_counts, w2_counts, ..., where w1_coutns means the appearaences of word w1 from the dictionary of the data set in the particular SMS message.

In [155]:
sms_df['SMS'] = sms_df['SMS'].str.replace(r'\W', ' ').str.replace(r'\s+',' ').str.lower()

In [156]:
sms_train.head(2)

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan


In [157]:
# Extract the vocabulary of our training dataset
vocabulary = []
for index in sms_train.index:
    words = sms_train.loc[index,'SMS'].split()
    vocabulary = vocabulary + words
vocabulary = list(set(vocabulary))
len(vocabulary)

7783

In [171]:
# transform the data frame into a new format, such that each word in the vocabulary is a column, 
# and its value are the counts of that word in the SMS message
# we create new data frame starting from a dictionary
word_counts_per_sms = {unique_word: [0] * len(sms_train['SMS']) for unique_word in vocabulary}

def get_new_sms_df_format(df, word_counts_per_sms):
    for index, sms in enumerate(df['SMS']):
        for word in sms.split():
            if word in word_counts_per_sms.keys():
                word_counts_per_sms[word][index] += 1
    # add the Label column to the data
    new_df = pd.DataFrame(word_counts_per_sms)
    new_df = pd.concat([df, new_df], axis=1)
    return new_df
    
training_set = get_new_sms_df_format(sms_train, word_counts_per_sms)
test_set = get_new_sms_df_format(sms_test, word_counts_per_sms)

## Build the algorithm
Here we will do the following:
* Calculate the spam and ham probability in the training set
* Calculate the total number of words in spam and ham sets
* Build the partial word probabilities for the algorithm

In [159]:
is_spam = (training_set['Label'] == 'spam')
is_ham = (training_set['Label'] == 'ham')
p_spam = is_spam.sum()/training_set.shape[0]
p_ham = is_ham.sum()/training_set.shape[0]
print('Sanity check: (p_spam + p_ham) = {0}'.format(p_spam+p_ham))

Sanity check: (p_spam + p_ham) = 1.0


In [160]:
n_w_spam = training_set[is_spam].iloc[:,2:].sum().sum()
n_w_ham = training_set[is_ham].iloc[:,2:].sum().sum()
n_w_vocabulary = len(vocabulary)
alpha = 1

### Initialize parameters for algorithm
* Build the partial probabilities of the words P(word|ham), P(word|spam) 

In [161]:
# build a probability dictionary for each word in the 
# dictionary using the training dataset
def get_word_probability_subset(wordcount_df, vocabulary, 
                                n_w_subset, alpha):
    p_words = {word:0 for word in vocabulary}
    n_w_vocabulary = len(vocabulary)
    
    counts_per_word = wordcount_df[vocabulary].sum(axis=0)
    for word in vocabulary:
        p_words[word] = (counts_per_word[word] + alpha)/(n_w_subset + 
                                                        alpha*n_w_vocabulary)
    return p_words

In [162]:
p_word_spam = get_word_probability_subset(training_set[is_spam], vocabulary, n_w_spam, alpha)
p_word_ham = get_word_probability_subset(training_set[is_ham], vocabulary, n_w_ham, alpha)

### Classify the message as "spam" or "ham"

In [163]:
import re

def classify(message, p_spam, p_ham, p_word_spam, p_word_ham):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    vocabulary = p_word_spam.keys()
    for word in message:
        if word in vocabulary:
            p_spam_given_message *= p_word_spam[word]
            p_ham_given_message *= p_word_ham[word]

    #print('P(Spam|message):', p_spam_given_message)
    #print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        #print('Label: Ham')
        prediction = 'ham'
    elif p_ham_given_message < p_spam_given_message:
        #print('Label: Spam')
        prediction = 'spam'
    else:
        #print('Equal proabilities, have a human classify this!')
        prediction = 'spam'
    return prediction

In [164]:
message1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
message2 = "Sounds good, Tom, then see u there"
prediction1 = classify(message1, p_spam, p_ham, p_word_spam, p_word_ham)
prediction2 = classify(message2, p_spam, p_ham, p_word_spam, p_word_ham)
print('message1: {0}, message2: {1}'.format(prediction1, prediction2))

message1: spam, message2: ham


## Test algorithm performance
We test the accuracy of the algorithm with the test dataset
We check the sensitivity and specifity of the algorithm.

In [207]:
correct = []
test_copy = sms_test.reset_index(drop=True)
predictions = []
for index, sms in enumerate(test_copy['SMS']):
    prediction = classify(sms, p_spam, p_ham, p_word_spam, p_word_ham)
    predictions.append(prediction)
test_copy['prediction'] = predictions
test_copy['correct'] = (test_copy['prediction']==test_copy['Label'])

In [213]:
accuracy = test_copy['correct'].sum()/test_copy.shape[0]
sensitivity = (test_copy.loc[test_copy['correct'],'prediction']=='spam').sum()/(test_copy['prediction']=='spam').sum()
specifity = (test_copy.loc[test_copy['correct'],'prediction']=='ham').sum()/(test_copy['prediction']=='ham').sum()
print('accuracy: {0}, sensitivity: {1}, specifity: {2}'.format(accuracy, sensitivity, specifity))

accuracy: 0.9874326750448833, sensitivity: 0.9586206896551724, specifity: 0.9917440660474717


## Conclusion
As we see, this simple algorithm is capable of performing rather well, and achieves an accuracy of 98.74%, with a 95% sensitivity and 99% specifity. This is great, for 100 SPAM SMS only 5 will go over the filter, moreover, from 100 correct SMS only a ~1% will be mistaken for SPAM. This is great, otherwise we might miss some really important SMS!